##### 175. Combine Two Tables

- Table: Person <br>
+-------------+---------+<br>
| Column Name | Type |<br>
+-------------+---------+<br>
| PersonId | int |<br>
| FirstName | varchar |<br>
| LastName | varchar |<br>
+-------------+---------+<br>
PersonId is the primary key column for this table.<br>
- Table: Address<br>
+-------------+---------+<br>
| Column Name | Type |<br>
+-------------+---------+<br>
| AddressId | int |<br>
| PersonId | int |<br>
| City | varchar |<br>
| State | varchar |<br>
+-------------+---------+<br>
AddressId is the primary key column for this table.<br>

##### Write a SQL query for a report that provides the following information for each person in the Person table, regardless if there is an address for each of those people: 
  - FirstName, LastName, City, State

In [0]:
cols_person = ['PersonId', 'FirstName', 'LastName']
cols_address = ['AddressId', 'PersonId', 'City', 'Country']

data_person = [(25, "Alice", "Sheron"),
               (30, "Bob", "San"),
               (35, "Charlie", "Angeles"),
               (45, "Samson", "Ignis")]

data_address = [('123AB', '25', 'Washington', 'USA'),
                ('345CC', '30', 'New York', 'USA'),
                ('888RD', '35', 'California', 'USA'),
                ('233HJ', '40', 'Beijing', 'China')]

df_person = spark.createDataFrame(data = data_person, schema = cols_person)
df_address = spark.createDataFrame(data = data_address, schema = cols_address)

In [0]:
df_report = df_person.join(df_address, 'PersonId', 'left').select("FirstName", "LastName", "City", "Country")

df_report.display()

FirstName,LastName,City,Country
Alice,Sheron,Washington,USA
Bob,San,New York,USA
Charlie,Angeles,California,USA
Samson,Ignis,null,null


##### 176. Second Highest Salary
Write a SQL query to get the second highest salary from the Employee table.<br>
+----+--------+<br>
| Id | Salary |<br>
+----+--------+<br>
| 1 | 100 |<br>
| 2 | 200 |<br>
| 3 | 300 |<br>
+----+--------+<br>

##### For example, given the above Employee table, the query should return 200 as the second highest salary. If there is no second highest salary, then the query should return null.<br>
+---------------------+<br>
| SecondHighestSalary |<br>
+---------------------+<br>
| 200 |<br>
+---------------------+<br>

In [0]:
df_sal = spark.createDataFrame(data = [(1, 100), (1, 200), (1, 300), (1, 400)], schema = ['Id', 'Salary'])
df_sal.display()

Id,Salary
1,100
1,200
1,300
1,400


In [0]:
from pyspark.sql.functions import dense_rank, col
from pyspark.sql.window import Window

windowSpec = Window.partitionBy('Id').orderBy('Salary')

df_2nd_highest_sal = (df_sal
                      .withColumn('rank', dense_rank().over(windowSpec))
                      .filter(col("rank") == 2)
                      .withColumnRenamed("Salary", "SecondHighestSalary")
                      .drop("Id")
                      .drop("rank")
                      )

df_2nd_highest_sal.display()

SecondHighestSalary
200


##### 177. Nth Highest Salary

##### Write a SQL query to get the nth highest salary from the Employee table. <br>
+----+--------+<br>
| Id | Salary |<br>
+----+--------+<br>
| 1 | 100 |<br>
| 2 | 200 |<br>
| 3 | 300 |<br>
+----+--------+<br>
For example, given the above Employee table, the nth highest salary where n = 2 is 200. If there is no nth highest salary, then the query should return null.<br>
+------------------------+<br>
| getNthHighestSalary(2) |<br>
+------------------------+<br>
| 200 |<br>
+------------------------+<br>

In [0]:
def nthHighestSalary(df, n):
  df_nth_highest_sal = (df_sal
                      .withColumn('rank', dense_rank().over(windowSpec))
                      .filter(col("rank") == n)
                      .withColumnRenamed("Salary", f"{n}th_HighestSalary")
                      .drop("Id")
                      .drop("rank")
                      )
  return df_nth_highest_sal

In [0]:
n = 5
display(nthHighestSalary(df_sal, n))

5th_HighestSalary


In [0]:
n = 3
display(nthHighestSalary(df_sal, n))

3th_HighestSalary
300


##### 178. Rank Scores

##### Write a SQL query to rank scores. If there is a tie between two scores, both should have the same ranking. Note that after a tie, the next ranking number should be the next consecutive integer value. In other words, there should be no "holes" between ranks. <br>
+----+-------+<br>
| Id | Score |<br>
+----+-------+<br>
| 1 | 3.50 |<br>
| 2 | 3.65 |<br>
| 3 | 4.00 |<br>
| 4 | 3.85 |<br>
| 5 | 4.00 |<br>
| 6 | 3.65 |<br>
+----+-------+<br>
For example, given the above Scores table, your query should generate the following report (order by highest score):<br>
+-------+------+<br>
| Score | Rank |<br>
+-------+------+<br>
| 4.00 | 1 |<br>
| 4.00 | 1 |<br>
| 3.85 | 2 |<br>
| 3.65 | 3 |<br>
| 3.65 | 3 |<br>
| 3.50 | 4 |<br>
+-------+------+<br>

In [0]:
df_score = (spark
            .createDataFrame(
              data = [(1, 3.50), (2, 3.65), (3, 4.00), (4, 3.85), (5, 4.00), (6, 3.65)],
              schema = ['Id', 'Score']
              )
            )

# df_score.display()

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank

scoreWindow = Window.orderBy(col('Score').desc())

df_score_rank = (df_score
                 .withColumn('Rank', dense_rank().over(scoreWindow))
                 .drop('Id')
                 )
                 
df_score_rank.display()

Score,Rank
4.0,1
4.0,1
3.85,2
3.65,3
3.65,3
3.5,4


##### 180. Consecutive Numbers

##### Write a SQL query to find all numbers that appear at least three times consecutively. <br>
+----+-----+<br>
| Id | Num |<br>
+----+-----+<br>
| 1 | 1 |<br>
| 2 | 1 |<br>
| 3 | 1 |<br>
| 4 | 2 |<br>
| 5 | 1 |<br>
| 6 | 2 |<br>
| 7 | 2 |<br>
+----+-----+<br>
For example, given the above Logs table, 1 is the only number that appears consecutively for at least three times.<br>
+-----------------+<br>
| ConsecutiveNums |<br>
+-----------------+<br>
| 1 |<br>
+-----------------+<br>

In [0]:
df_num = (spark
          .createDataFrame(
            data = [(1,1), (2,1), (3,1), (4,2), (5,2), (6,2), (7,3), (8,1), (9,3)],
            schema = ['Id', 'Num']
            )
          )

# df_num.display()

In [0]:
from pyspark.sql.functions import lead

df_num_lead = (df_num
               .withColumn("Next_num", lead('Num', 1).over(Window.orderBy('Id')))
               .withColumn("Next_to_next_num", lead('Next_num', 1).over(Window.orderBy('Id')))
               )

df_num_lead.display()

df_num_most = (df_num_lead
               .filter((col("Num") == col("Next_num")) & (col("Next_num") == col("Next_to_next_num")))
               .select(col("Num").alias("ConsecutiveNums"))
               )

df_num_most.display()

Id,Num,Next_num,Next_to_next_num
1,1,1,1
2,1,1,2
3,1,2,2
4,2,2,2
5,2,2,3
6,2,3,1
7,3,1,3
8,1,3,null
9,3,null,null


ConsecutiveNums
1
2


##### 181. Employees Earning More Than Their Managers

The Employee table holds all employees including their managers. Every employee has an Id, and there is also a column for the manager Id.<br>
+----+-------+--------+-----------+<br>
| Id | Name | Salary | ManagerId |<br>
+----+-------+--------+-----------+<br>
| 1 | Joe | 70000 | 3 |<br>
| 2 | Henry | 80000 | 4 |<br>
| 3 | Sam | 60000 | NULL |<br>
| 4 | Max | 90000 | NULL |<br>
+----+-------+--------+-----------+<br>

##### Given the Employee table, write a SQL query that finds out employees who earn more than their managers. For the above table, Joe is the only employee who earns more than his manager.<br>
+----------+<br>
| Employee |<br>
+----------+<br>
| Joe |<br>
+----------+<br>

In [0]:
df_emp = (spark
          .createDataFrame(
            data = [(1, 'Joe', 70000, 3),
                    (2, 'Henry', 80000, 4),
                    (3, 'Sam', 60000, None),
                    (4, 'Max', 90000, None)
                    ],
            schema = ['Id', 'Name', 'Salary', 'ManagerId']
            )
          )

# display(df_emp)

In [0]:
from pyspark.sql.functions import col

df_emp_man = (df_emp.alias("emp")
              .join(df_emp.alias("manager"),
                    col("emp.ManagerId") == col("manager.Id"),
                    'inner'
                    )
              .filter(col("emp.Salary") > col("manager.Salary"))
              .select(col("emp.Name"))
              )

df_emp_man.display()

Name
Joe
